## Домашнее задание 
#### 1. Побейте метрику на лидерборде map@10 = 0.075 c моделью из implicit, lightfm или rectools, в том числе используя ANN **(5 баллов)**
#### 2. Реализуйте эксперименты с кастомной моделю kNN с семинара. Результат - ноутбук(и) **(максимум 12 баллов)**
Что можно сделать в ноутбуке:
- Реализовать тюнинг гиперпараметров для моделей из implicit, lightfm или rectools **(3 балла)**
  - Для перебора гиперпараметров можно использовать [`Optuna`](https://github.com/optuna/optuna), [`Hyperopt`](https://github.com/hyperopt/hyperopt)
- Воспользоваться методом приближенного поиска соседей для выдачи рекомендаций. **(3 балла)**
    - Можно использовать любые удобные: [`Annoy`](https://github.com/spotify/annoy), [`nmslib`](https://github.com/nmslib/nmslib) и.т.д
- Сделать рекомендации для холодных пользователей используя их фичи (для кого нет фичей - там другим способом) **(3 балла)**

  ------------------------------------

In [1]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"  # For implicit ALS

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
!pip install optuna -q

In [84]:
!pip install dill -q

In [85]:
import pandas as pd
import numpy as np

from implicit.als import AlternatingLeastSquares

from rectools.metrics import Precision, Recall, MAP, MRR,  MeanInvUserFreq, Serendipity, calc_metrics
from rectools.models import PopularModel, RandomModel, ImplicitALSWrapperModel
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel

import matplotlib.pyplot as plt
import seaborn as sns
import optuna

import matplotlib.pyplot as plt
import dill
import pickle
from pathlib import Path
import typing as tp
from tqdm import tqdm

from lightfm import LightFM

from implicit.bpr import BayesianPersonalizedRanking

from implicit.lmf import LogisticMatrixFactorization

In [83]:
DATA_PATH = Path("data_original")

# Load Data

In [122]:
%%time
users = pd.read_csv(DATA_PATH / 'users.csv')
items = pd.read_csv(DATA_PATH / 'items.csv')
interactions = pd.read_csv(DATA_PATH / 'interactions.csv')

CPU times: total: 1.94 s
Wall time: 3.6 s


In [123]:
Columns.Datetime = 'last_watch_dt'
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')
max_date = interactions[Columns.Datetime].max()

In [124]:
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)

In [125]:
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()

print(f"train: {train.shape}")
print(f"test: {test.shape}")

train: (4985269, 6)
test: (490982, 6)


In [126]:
# фильтруем тех кто посмотрел меньше 300 секунд
train.drop(train.query("total_dur < 300").index, inplace=True)

In [127]:
# фильтруем холодных пользователей из теста
cold_users = set(test[Columns.User]) - set(train[Columns.User])

In [130]:
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

# Features

In [131]:
users.isnull().sum()

user_id         0
age         14095
income      14776
sex         13831
kids_flg        0
dtype: int64

In [132]:
users.fillna('Unknown', inplace=True)

In [133]:
users.nunique()

user_id     840197
age              7
income           7
sex              3
kids_flg         2
dtype: int64

In [134]:
users = users.loc[users[Columns.User].isin(train[Columns.User])].copy()
users

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
5,1037719,age_45_54,income_60_90,М,0
...,...,...,...,...,...
840184,529394,age_25_34,income_40_60,Ж,0
840186,80113,age_25_34,income_40_60,Ж,0
840188,312839,age_65_inf,income_60_90,Ж,0
840189,191349,age_45_54,income_40_60,М,1


In [135]:
user_features_frames = []
for feature in ["sex", "age", "income"]:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features.head()

,id,value,feature
0,973171,М,sex
1,962099,М,sex
3,721985,Ж,sex
4,704055,Ж,sex
5,1037719,М,sex


## Items

In [136]:
items.isnull().sum()

item_id             0
content_type        0
title               0
title_orig       4745
release_year       98
genres              0
countries          37
for_kids        15397
age_rating          2
studios         14898
directors        1509
actors           2619
description         2
keywords          423
dtype: int64

In [137]:
items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()
items.head()

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


In [138]:
items.nunique()

item_id         14019
content_type        2
title           13454
title_orig       9724
release_year      104
genres           2559
countries         666
for_kids            2
age_rating          6
studios            38
directors        7414
actors          11830
description     13791
keywords        13583
dtype: int64

### Genre

In [139]:
# Explode genres to flatten table
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


In [140]:
genre_feature["value"].value_counts()

драмы                 4923
комедии               3479
зарубежные            3055
мелодрамы             2533
триллеры              2297
                      ... 
токшоу                   2
красота и здоровье       2
передачи                 1
образование              1
рекомендуем              1
Name: value, Length: 94, dtype: int64

### Content

In [141]:
content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"
content_feature

,id,value,feature
0,10711,film,content_type
1,2508,film,content_type
2,10716,film,content_type
3,7868,film,content_type
4,16268,film,content_type
...,...,...,...
15958,6443,series,content_type
15959,2367,series,content_type
15960,10632,series,content_type
15961,4538,series,content_type


In [142]:
content_feature['value'].value_counts()

film      10662
series     3357
Name: value, dtype: int64

In [143]:
item_features = pd.concat((genre_feature, content_feature))
item_features

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre
...,...,...,...
15958,6443,series,content_type
15959,2367,series,content_type
15960,10632,series,content_type
15961,4538,series,content_type


# Hyperparameters
- Реализовать тюнинг гиперпараметров для моделей из implicit, lightfm или rectools **(3 балла)**
  - Для перебора гиперпараметров буду использовать [`Optuna`](https://github.com/optuna/optuna)

Буду использовать модели, которые были на практике из ректулз
- ALS
- LightFM

In [144]:
K_RECOS = 10
RANDOM_STATE = 42
NUM_THREADS = 16
N_TRIALS = 15
N_EPOCHS = 1 # Lightfm
USER_ALPHA = 0 # Lightfm
ITEM_ALPHA = 0 # Lightfm

In [27]:
%%time
dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

CPU times: total: 234 ms
Wall time: 667 ms


In [28]:
TEST_USERS = test[Columns.User].unique()

Выберем метрику MAP для подбора гиперпараметров, так как нам именно ее и надо побить на лидерборде
is_fitting_features будет константно тру, так как на практике нигде не было чтобы False дал лучший результат

**Для начала обучим ALS**

In [60]:
def objective(trial):
    n_factors = trial.suggest_int('components', 16, 64, step=16)
    lr = trial.suggest_float('lr', 1e-4, 1e-1, log = True)
    
    als_model = ImplicitALSWrapperModel(
        AlternatingLeastSquares(
            factors=n_factors, 
            random_state=RANDOM_STATE, 
            num_threads=NUM_THREADS,
        ),
        fit_features_together=True,
    )
    als_model.fit(dataset)

    recos_als = als_model.recommend(
        users=TEST_USERS,
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
    )
    
    map10 = MAP(k=K_RECOS).calc(recos_als, test)
    return map10

In [61]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=N_TRIALS)

print("Finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-01-17 13:28:28,107] A new study created in memory with name: no-name-3bfa079f-7faf-463b-9de5-4bd2d4d0e099
[I 2024-01-17 13:28:50,139] Trial 0 finished with value: 0.07464975414579311 and parameters: {'components': 32, 'lr': 0.005042597970694257}. Best is trial 0 with value: 0.07464975414579311.
[I 2024-01-17 13:29:10,800] Trial 1 finished with value: 0.07510425594922608 and parameters: {'components': 16, 'lr': 0.05942215468443357}. Best is trial 1 with value: 0.07510425594922608.
[I 2024-01-17 13:29:32,805] Trial 2 finished with value: 0.07464975414579311 and parameters: {'components': 32, 'lr': 0.07533229352477559}. Best is trial 1 with value: 0.07510425594922608.
[I 2024-01-17 13:29:54,316] Trial 3 finished with value: 0.07510425594922608 and parameters: {'components': 16, 'lr': 0.00017440006157014014}. Best is trial 1 with value: 0.07510425594922608.
[I 2024-01-17 13:30:15,938] Trial 4 finished with value: 0.07510425594922608 and parameters: {'components': 16, 'lr': 0.040034

Finished trials: 15
Best trial:
  Value: 0.07510425594922608
  Params: 
    components: 16
    lr: 0.05942215468443357


**Теперь посмотрим на модель из LightFM**

Заранее выбрала Loss Warp, потому что из практики стало ясно, что он больше всего подходит в нашей задаче

In [74]:
def objective(trial):
    n_factors = trial.suggest_int('components', 16, 64, step=16)
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log = True)
    
    lightfm_model = LightFMWrapperModel(
            LightFM(
                no_components=n_factors, 
                loss='warp', 
                random_state=RANDOM_STATE,
                learning_rate=lr,
                user_alpha=USER_ALPHA,
                item_alpha=ITEM_ALPHA,
            ),
            epochs=N_EPOCHS,
            num_threads=NUM_THREADS,
        )
    
    
    lightfm_model.fit(dataset)

    recos_lFM = lightfm_model.recommend(
        users=TEST_USERS,
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
    )
    
    map10 = MAP(k=K_RECOS).calc(recos_lFM, test)
    return map10

In [75]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=N_TRIALS)

print("Finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-01-17 13:46:47,611] A new study created in memory with name: no-name-28aa90d5-3a85-43d9-b768-89ca2f26678d
[I 2024-01-17 13:47:53,603] Trial 0 finished with value: 0.05430624508023497 and parameters: {'components': 48, 'lr': 0.00016109402320245298}. Best is trial 0 with value: 0.05430624508023497.
[I 2024-01-17 13:48:30,265] Trial 1 finished with value: 0.07677755691902213 and parameters: {'components': 32, 'lr': 0.003138045529061502}. Best is trial 1 with value: 0.07677755691902213.
[I 2024-01-17 13:49:58,962] Trial 2 finished with value: 0.0004584443098254922 and parameters: {'components': 64, 'lr': 2.272764361152419e-05}. Best is trial 1 with value: 0.07677755691902213.
[I 2024-01-17 13:50:33,923] Trial 3 finished with value: 0.07727909442366546 and parameters: {'components': 32, 'lr': 0.005553300236049671}. Best is trial 3 with value: 0.07727909442366546.
[I 2024-01-17 13:51:32,098] Trial 4 finished with value: 0.07739908174951539 and parameters: {'components': 64, 'lr': 0.0

Finished trials: 15
Best trial:
  Value: 0.08000534197320383
  Params: 
    components: 48
    lr: 0.009914640727824435


Выберем лучшую модель, чтобы сохранить ее веса и обучим на всем датасете Киона, чтобы можно было давать для всех пользователей хорошие рекомендации непосредственно в сервисе моем

Поэтому повторяем шаги выше, но для всего датасета, а не трейна (isin(interactions[Columns.User])])

In [77]:
users_all = users.loc[users[Columns.User].isin(interactions[Columns.User])]

user_features_frames = []
for feature in ["sex", "age", "income"]:
    feature_frame = users_all.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)

In [78]:
items_all = items.loc[items[Columns.Item].isin(interactions[Columns.Item])]

items_all["genre"] = items_all["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items_all[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

content_feature = items_all.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"

item_features = pd.concat((genre_feature, content_feature))

In [79]:
kion_dataset = Dataset.construct(
    interactions_df=interactions,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

In [86]:
best_model = LightFMWrapperModel(
    LightFM(
        no_components=48,
        loss='warp',
        learning_rate=0.0099,
        random_state=RANDOM_STATE
    ),
    num_threads=NUM_THREADS,
)
    
best_model.fit(kion_dataset)


In [87]:
with open('LightFM_warp_48.dill', 'wb') as f:
    dill.dump(best_model, f)

Мой сервис будет оффлайн, поэтому давайте выдадим рекомендации, а потом скачаем даные в pkl

In [88]:
recos = best_model.recommend(
    users=interactions['user_id'].unique(),
    dataset=kion_dataset,
    k=K_RECOS,
    filter_viewed=True,
)

In [89]:
recos = recos[['user_id', 'item_id']].groupby('user_id')['item_id'].apply(list).T.to_dict()
with open('offline_lightfm.pkl', 'wb') as f:
    pickle.dump(recos, f)

# ANN (Approximate Nearest Neighbors) 

Воспользоваться методом приближенного поиска соседей для выдачи рекомендаций. **(3 балла)**

Буду использовать nmsl

Подготовим данные как в практике

In [106]:
import time
import nmslib

In [91]:
best_model

In [96]:
user_embeddings, item_embeddings = best_model.get_vectors(kion_dataset)
user_embeddings.shape, item_embeddings.shape

((962179, 50), (15706, 50))

In [97]:
def augment_inner_product(factors):
    normed_factors = np.linalg.norm(factors, axis=1)
    max_norm = normed_factors.max()
    
    extra_dim = np.sqrt(max_norm ** 2 - normed_factors ** 2).reshape(-1, 1)
    augmented_factors = np.append(factors, extra_dim, axis=1)
    return max_norm, augmented_factors

In [98]:
print('pre shape: ', item_embeddings.shape)
max_norm, augmented_item_embeddings = augment_inner_product(item_embeddings)
augmented_item_embeddings.shape

pre shape:  (15706, 50)


(15706, 51)

In [99]:
extra_zero = np.zeros((user_embeddings.shape[0], 1))
augmented_user_embeddings = np.append(user_embeddings, extra_zero, axis=1)
augmented_user_embeddings.shape

(962179, 51)

In [101]:
item_id = 0
item_embeddings[item_id]

array([ 1.        ,  1.36008537,  0.25023073, -0.1402979 , -0.1401263 ,
       -0.28630489, -0.19733539,  0.00412825, -0.11233127,  0.4974274 ,
        0.49853653, -0.42191634, -0.39138407, -0.32965979, -0.32010901,
       -0.0728093 ,  0.2805641 , -0.04958308, -0.03296055,  0.28894499,
        0.08770996, -0.23827758,  0.45104712,  0.46520001,  0.22159365,
        0.31081152,  0.50805128, -0.2289442 ,  0.24647547,  0.14184225,
        0.00181587, -0.02281912,  0.66063768, -0.29613733, -0.15686691,
       -0.56488705,  0.11684057, -0.39452374,  0.19799615, -0.21081914,
       -0.04885705, -0.43194729,  0.05919545, -0.38106644,  0.22361603,
       -0.04118441,  0.15361363,  0.11481205,  0.02631835, -0.21486877])

In [102]:
augmented_item_embeddings[item_id]

array([ 1.00000000e+00,  1.36008537e+00,  2.50230730e-01, -1.40297905e-01,
       -1.40126303e-01, -2.86304891e-01, -1.97335392e-01,  4.12824750e-03,
       -1.12331271e-01,  4.97427404e-01,  4.98536527e-01, -4.21916336e-01,
       -3.91384065e-01, -3.29659790e-01, -3.20109010e-01, -7.28093013e-02,
        2.80564100e-01, -4.95830812e-02, -3.29605527e-02,  2.88944989e-01,
        8.77099633e-02, -2.38277584e-01,  4.51047122e-01,  4.65200007e-01,
        2.21593648e-01,  3.10811520e-01,  5.08051276e-01, -2.28944197e-01,
        2.46475473e-01,  1.41842246e-01,  1.81587040e-03, -2.28191167e-02,
        6.60637677e-01, -2.96137333e-01, -1.56866908e-01, -5.64887047e-01,
        1.16840571e-01, -3.94523740e-01,  1.97996154e-01, -2.10819140e-01,
       -4.88570482e-02, -4.31947291e-01,  5.91954514e-02, -3.81066442e-01,
        2.23616034e-01, -4.11844105e-02,  1.53613627e-01,  1.14812046e-01,
        2.63183527e-02, -2.14868769e-01,  2.46384924e+00])

In [104]:
# Set index parameters
# These are the most important ones
M = 48
efC = 100

num_threads = 4
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC, 'post' : 0}
print('Index-time parameters', index_time_params)

# Number of neighbors 
K=10

# Space name should correspond to the space name 
# used for brute-force search
space_name='negdotprod'

Index-time parameters {'M': 48, 'indexThreadQty': 4, 'efConstruction': 100, 'post': 0}


In [107]:
# Intitialize the library, specify the space, the type of the vector and add data points 
index = nmslib.init(method='hnsw', space=space_name, data_type=nmslib.DataType.DENSE_VECTOR) 
index.addDataPointBatch(augmented_item_embeddings) 

15706

In [108]:
# Create an index
start = time.time()
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC}
index.createIndex(index_time_params) 
end = time.time() 
print('Index-time parameters', index_time_params)
print('Indexing time = %f' % (end-start))

Index-time parameters {'M': 48, 'indexThreadQty': 4, 'efConstruction': 100}
Indexing time = 0.161812


In [109]:
# Setting query-time parameters
efS = 100
query_time_params = {'efSearch': efS}
print('Setting query-time parameters', query_time_params)
index.setQueryTimeParams(query_time_params)

Setting query-time parameters {'efSearch': 100}


In [110]:
# Querying
query_qty = augmented_user_embeddings.shape[0]
start = time.time() 
nbrs = index.knnQueryBatch(augmented_user_embeddings, k = K, num_threads = num_threads)
end = time.time() 
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty)) 

kNN time total=5.545326 (sec), per query=0.000006 (sec), per query adjusted for thread number=0.000023 (sec)


In [111]:
nbrs[0]

(array([ 32,  16,  25,  84, 235,  10,  21, 174,  68, 142]),
 array([42.023033, 42.034996, 42.409637, 42.454338, 42.6675  , 42.66977 ,
        42.699825, 42.756584, 42.862514, 42.883026], dtype=float32))

Скачаем решение ANN

In [116]:
all_users = interactions['user_id'].unique()
len(all_users)

962179

In [115]:
len(nbrs)

962179

In [117]:
recos = {all_users[i]: list(nbrs[i][0]) for i in range(len(nbrs))}

In [118]:
with open('ANN_nmslib_lightfm_48.pkl', 'wb') as f:
    pickle.dump(recos, f)

# Cold users

Сделать рекомендации для холодных пользователей используя их фичи (для кого нет фичей - там другим способом) **(3 балла)**

Для холодных юзеров буду использовать популярное как и в прошлый раз. Для теплых будем делать предсказание по соседям. Это все реализую в сервисе